In [44]:
import json
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt

# Load data from the JSON file
with open('Kidney_Transplant_Orchestrator.json', 'r') as file:
    data = json.load(file)

# Compatibility based on blood type
donor_recipient_compatibility = {
    'O': ['O', 'A', 'B', 'AB'],
    'A': ['A', 'AB'],
    'B': ['B', 'AB'],
    'AB': ['AB']
}

In [45]:
# Create directed graph
G = nx.DiGraph()

# Add nodes and edges based on compatibility
counter = 0
for element in data:
    G.add_nodes_from([(counter, element)])
    counter += 1

# Create edges based on donor-recipient compatibility
for node in G.nodes:
    donors = G.nodes[node]['Donor']
    for donor in donors:
        for vertex in G.nodes:
            if node == vertex:
                continue
            recipient = G.nodes[vertex]['Recipient']
            if recipient in donor_recipient_compatibility[donor]:
                G.add_edge(node, vertex)

print('Total edges:', len(G.edges))

Total edges: 171996


In [46]:
#-------------------------------------------------------PARTICIPANT-MAXIMIZING PLAN----------------------------------------------------------------------

In [47]:
# ----------- FINDING 2-WAY AND 3-WAY CYCLES  ------------------

def find_2_and_3_way_cycles(G):
    """
    Finds all unique 2-way and 3-way cycles in the graph G.
    """
    cycle_2 = []
    cycle_3 = []
    H = G.to_undirected()  # Convert to undirected graph for neighbor checks

    # Initialize visited status for each directed edge
    for (i, j) in G.edges:
        G.edges[(i, j)]["visited"] = 0

    # Detect 3-way cycles
    for (i, j) in H.edges:
        for k in nx.common_neighbors(H, i, j):
            # Check if (i, j), (j, k), (k, i) form a valid 3-way cycle
            if (i, j) in G.edges and G.edges[(i, j)]["visited"] < 2:
                if (j, k) in G.edges and G.edges[(j, k)]["visited"] < 2:
                    if (k, i) in G.edges and G.edges[(k, i)]["visited"] < 2:
                        cycle = tuple(sorted([i, j, k]))  # Sort nodes to ensure uniqueness
                        if cycle not in cycle_3:  # Ensure no duplicate cycles
                            cycle_3.append(cycle)
                            # Mark edges as visited
                            G.edges[(i, j)]["visited"] += 1
                            G.edges[(j, k)]["visited"] += 1
                            G.edges[(k, i)]["visited"] += 1

    # Reset visited status for 2-way cycle detection
    for (i, j) in G.edges:
        G.edges[(i, j)]["visited"] = False

    # Detect 2-way cycles
    for (i, j) in G.edges:
        if G.edges[(i, j)]["visited"]:
            continue  # Skip if already visited
        if (j, i) in G.edges:  # Check if reciprocal edge exists
            cycle_2.append((i, j))  # Add as a 2-way cycle
            # Mark both edges as visited
            G.edges[(i, j)]["visited"] = True
            G.edges[(j, i)]["visited"] = True

    # Combine both lists into a single list of cycles
    return cycle_2 + cycle_3

In [48]:
# ----------- GUROBI OPTIMIZATION ----------------
# Combine 2-way and 3-way cycles 
cycles = find_2_and_3_way_cycles(G)

# Create a Gurobi model
m = gp.Model("Kidney_Exchange")

# Add binary decision variables for each cycle (2-way or 3-way)
x = m.addVars(cycles, vtype=GRB.BINARY, name="x")

# Objective: Maximize the number of participants in selected cycles
m.setObjective(gp.quicksum(x[c] * (2 if len(c) == 2 else 3) for c in cycles), GRB.MAXIMIZE)

# Constraints: Each node (donor or recipient) can participate in at most one cycle
m.addConstrs(gp.quicksum(x[c] for c in cycles if v in c) <= 1 for v in G.nodes)

# Optimize the model
m.optimize()

# Display results
if m.SolCount > 0:
    print(f"\nOptimal Solution Found:")
    print(f"Total Participants in Selected Cycles (Objective Value): {m.objVal}")

    selected_cycles = [c for c in cycles if x[c].X > 0.5]
    print("\nSelected Cycles (Donor-Recipient Pairs):")
    for cycle in selected_cycles:
        if len(cycle) == 2:
            print(f"2-Way Cycle: Donor {cycle[0]} <-> Recipient {cycle[1]}")
        elif len(cycle) == 3:
            print(f"3-Way Cycle: Donor {cycle[0]} -> Recipient {cycle[1]} -> Donor {cycle[1]} -> Recipient {cycle[2]} -> Donor {cycle[2]} -> Recipient {cycle[0]}")
else:
    print("No feasible solution found.")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1140G7 @ 1.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 715 rows, 32448 columns and 64896 nonzeros
Model fingerprint: 0xd91fd067
Variable types: 0 continuous, 32448 integer (32448 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 338.0000000
Presolve removed 354 rows and 0 columns
Presolve time: 0.09s
Presolved: 361 rows, 32448 columns, 64896 nonzeros
Variable types: 0 continuous, 32448 integer (32448 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity     -7.2000000e+02      0s
       1        422  -2.0000

In [49]:
#----------------------------------------------------------MATCH-MAXIMIZING PLAN----------------------------------------------------------------------

In [50]:
def find_2_and_3_way_cycles(G):
    """
    Finds valid 2-way and 3-way cycles in the graph G.
    Returns a list of tuples representing each cycle.
    """
    cycles = []

    # Find 2-way cycles
    for (i, j) in G.edges:
        if G.has_edge(j, i) and i < j:  # Ensure no duplicates by ordering nodes
            cycles.append((i, j))

    # Find 3-way cycles
    for i in G.nodes:
        for j in G.successors(i):
            if j == i:
                continue  # Avoid self-loops
            for k in G.successors(j):
                if k == i or k == j:
                    continue  # Avoid self-loops and repeated nodes
                if G.has_edge(k, i):  # Check if it completes a 3-way cycle
                    # Only add unique 3-way cycles (sorted order)
                    cycles.append(tuple(sorted((i, j, k))))

    # Remove duplicate 3-way cycles
    cycles = list(set(cycles))
    return cycles

In [51]:
# ----------- GUROBI OPTIMIZATION ----------------
# Combine 2-way and 3-way cycles 
cycles = find_2_and_3_way_cycles(G)

# Create a Gurobi model
m = gp.Model("Kidney_Exchange")

# Add binary decision variables for each cycle (2-way or 3-way)
x = m.addVars(cycles, vtype=GRB.BINARY, name="x")

# Alternative Objective: Maximize the number of selected cycles (matches)
m.setObjective(gp.quicksum(x[c] for c in cycles), GRB.MAXIMIZE)


# Constraints: Each node (donor or recipient) can participate in at most one cycle
m.addConstrs(gp.quicksum(x[c] for c in cycles if v in c) <= 1 for v in G.nodes)

# Optimize the model
m.optimize()

# Display results
if m.SolCount > 0:
    print(f"\nOptimal Solution Found:")
    print(f"Total Participants in Selected Cycles (Objective Value): {m.objVal}")

    selected_cycles = [c for c in cycles if x[c].X > 0.5]
    print("\nSelected Cycles (Donor-Recipient Pairs):")
    for cycle in selected_cycles:
        if len(cycle) == 2:
            print(f"2-Way Cycle: Donor {cycle[0]} <-> Recipient {cycle[1]}")
        elif len(cycle) == 3:
            print(f"3-Way Cycle: Donor {cycle[0]} -> Recipient {cycle[1]} -> Donor {cycle[1]} -> Recipient {cycle[2]} -> Donor {cycle[2]} -> Recipient {cycle[0]}")
else:
    print("No feasible solution found.")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1140G7 @ 1.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 715 rows, 32448 columns and 64896 nonzeros
Model fingerprint: 0xb3379e69
Variable types: 0 continuous, 32448 integer (32448 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 169.0000000
Presolve removed 354 rows and 0 columns
Presolve time: 0.11s
Presolved: 361 rows, 32448 columns, 64896 nonzeros
Variable types: 0 continuous, 32448 integer (32448 binary)

Root relaxation: cutoff, 1083 iterations, 0.05 seconds (0.06 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Ga